In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.8 MB/s eta 0:00:00


In [ ]:
# Load and preprocess the data
import pandas as pd
df = pd.read_csv("/content/gdrive/MyDrive/Capstone Project /cleaned.csv")
df.head()

,article,highlights,cleaned_text
0,by . associated press . published: . 14:11 est...,bishop john folda north dakota taking time dia...,associated press published est october updated...
1,(cnn) -- ralph mata was an internal affairs li...,criminal complaint cop used role help cocaine ...,cnn ralph mata internal affair lieutenant miam...
2,a drunk driver who killed a young woman in a h...,craig ecclestontodd drunk least three pint dri...,drunk driver killed young woman headon crash c...
3,(cnn) -- with a breezy sweep of his pen presid...,nina do santos say europe must ready accept sa...,cnn breezy sweep pen president vladimir putin ...
4,fleetwood are the only team still to have a 10...,fleetwood top league one win scunthorpe peterb...,fleetwood team still record sky bet league one...


In [ ]:
# create a subset of first 50,000 rows
subset = df.head(50000)

df = subset

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   article       50000 non-null  object
 1   highlights    50000 non-null  object
 2   cleaned_text  50000 non-null  object
dtypes: object(3)
memory usage: 1.1+ MB


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('t5-base')
model = AutoModelWithLMHead.from_pretrained('t5-base', return_dict=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language

In [ ]:
inputs = tokenizer.encode("summarize: " + df['cleaned_text'][0],
return_tensors='pt',
max_length=512,
truncation=True)

In [ ]:
summary_ids = model.generate(inputs, max_length=150, min_length=80, length_penalty=5., num_beams=2)

In [ ]:
summary = tokenizer.decode(summary_ids[0])

In [ ]:
df['cleaned_text'][0]

'associated press published est october updated est october bishop fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis virus late september early october state health department issued advisory exposure anyone attended five church took communion bishop john folda pictured fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis state immunization program manager molly howell say risk low official feel important alert people possible exposure diocese announced monday bishop john folda taking time diagnosed hepatitis diocese say contracted infection contaminated food attending conference newly ordained bishop italy last month symptom hepatitis include fever tiredness loss appetite nausea abdominal discomfort fargo catholic diocese north dakota pictured bishop located'

In [ ]:
df['highlights'][0]

'bishop john folda north dakota taking time diagnosed contracted infection contaminated food italy church member fargo grand fork jamestown could exposed'

In [ ]:
summary

'<pad> bishop fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis virus late september early october state health department issued advisory exposure anyone attended five church took communion. bishop john folda taking time diagnosed hepatitis diocese say contracted infection contaminated food attending conference newly ordained bishop italy last month symptom hepatitis include fever tiredness loss appetite nausea abdominal discomfort fargo catholic diocese say exposed</s>'

In [ ]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/



There are different ways to evaluate the accuracy of generated summaries. One common method is to use the ROUGE (Recall-Oriented Understudy for Gisting Evaluation) metrics. ROUGE measures the overlap between the generated summary and the reference (human-written) summary in terms of n-grams, where n can be 1, 2, or 3. The most commonly used ROUGE scores are ROUGE-1, ROUGE-2, and ROUGE-L (which measures the longest common subsequence between the generated and reference summaries).

In [ ]:
from rouge import Rouge

# Initialize the Rouge object
rouge = Rouge()

# Example of a generated summary and its reference summary
generated_summary = summary
reference_summary = df['cleaned_text'][0]

# Calculate ROUGE scores
scores = rouge.get_scores(generated_summary, reference_summary)

# Print the scores
print(scores)

[{'rouge-1': {'r': 0.7215189873417721, 'p': 0.9661016949152542, 'f': 0.8260869516267592}, 'rouge-2': {'r': 0.6666666666666666, 'p': 0.9696969696969697, 'f': 0.7901234519615913}, 'rouge-l': {'r': 0.7215189873417721, 'p': 0.9661016949152542, 'f': 0.8260869516267592}}]


In [ ]:
from tabulate import tabulate

# Assuming you have the scores for each document in a list named `scores`

table = [['Document', 'ROUGE-1', 'ROUGE-2', 'ROUGE-L']]
for i, s in enumerate(scores):
    table.append([f'Document {i+1}', s['rouge-1']['f'], s['rouge-2']['f'], s['rouge-l']['f']])

print(tabulate(table, headers='firstrow'))


Document      ROUGE-1    ROUGE-2    ROUGE-L
----------  ---------  ---------  ---------
Document 1   0.826087   0.790123   0.826087


In [ ]:
from tabulate import tabulate

scores = [{'rouge-1': {'r': 0.7215189873417721, 'p': 0.9661016949152542, 'f': 0.8260869516267592}, 'rouge-2': {'r': 0.6666666666666666, 'p': 0.9696969696969697, 'f': 0.7901234519615913}, 'rouge-l': {'r': 0.7215189873417721, 'p': 0.9661016949152542, 'f': 0.8260869516267592}}]

table = [['Metric', 'Recall', 'Precision', 'F1 Score']]
for key, value in scores[0].items():
    table.append([key.upper(), value['r'], value['p'], value['f']])

print(tabulate(table, headers='firstrow'))


Metric      Recall    Precision    F1 Score
--------  --------  -----------  ----------
ROUGE-1   0.721519     0.966102    0.826087
ROUGE-2   0.666667     0.969697    0.790123
ROUGE-L   0.721519     0.966102    0.826087
